# HW06
Input some other corpus of text input & see what awesome generated outputs you can come up with!
Based on http://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

The corpus Bible KJV is found from http://www.gutenberg.org/cache/epub/10/pg10.txt

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

In [2]:
PATH='data/bible/'

In [3]:
text = open(f'{PATH}bible_kjv_psalms.txt').read()
print('corpus length:', len(text))

corpus length: 240887


In [4]:
text[:400]

'\ufeffThe Book of Psalms\n\n\n1:1 Blessed is the man that walketh not in the counsel of the ungodly,\nnor standeth in the way of sinners, nor sitteth in the seat of the\nscornful.\n\n1:2 But his delight is in the law of the LORD; and in his law doth he\nmeditate day and night.\n\n1:3 And he shall be like a tree planted by the rivers of water, that\nbringeth forth his fruit in his season; his leaf also shall not\nw'

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 75


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

''.join(chars[1:-6])

"\n !'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstu"

Map from chars to indices and back again

In [7]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [8]:
idx = [char_indices[c] for c in text]

idx[:10]

[74, 42, 55, 52, 2, 24, 62, 62, 58, 2]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'\ufeffThe Book of Psalms\n\n\n1:1 Blessed is the man that walketh not in the c'

## Stateful model

### Setup

In [10]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/bible/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

bible_kjv_psalms.txt  bible_kjv.txt  models/  trn/  val/


In [11]:
with open(f'{PATH}bible_kjv_psalms.txt') as f:
    text = f.readlines()
    text_line_length = len(text)
    trn_index = int(text_line_length*.8)
    trn = text[:trn_index]
    tst = text[trn_index:]
    with open(f'{TRN}trn.txt','w') as f2:
        f2.writelines(trn)
    with open(f'{VAL}val.txt','w') as f3:
        f3.writelines(tst)  

In [12]:
%ls {PATH}trn

trn.txt


In [13]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=100; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(28, 46, 1, 188965)

### Putting it all together: LSTM

In [14]:
from fastai import sgdr

n_hidden=512

In [15]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

## LSTM 2 Layer

In [56]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)
os.makedirs(f'{PATH}models', exist_ok=True)

In [57]:
fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      3.151018   3.110161  
    1      3.092948   3.101662                            



[3.1016617]

In [58]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm2_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      3.053523   3.098494  
    1      3.032927   3.009468                            
    2      2.982719   2.958015                            
    3      2.922333   2.791516                            
    4      2.806965   2.567846                            
    5      2.637148   2.361142                            
    6      2.503784   2.323076                            
    7      2.42334    2.183152                            
    8      2.272974   1.996996                            
    9      2.119345   1.888866                            
    10     1.983422   1.781268                            
    11     1.869926   1.711974                            
    12     1.783656   1.683709                            
    13     1.719857   1.659415                            
    14     1.677643   1.652955                            



[1.6529553]

In [59]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm2_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      1.634653   1.6348    
    1      1.60697    1.594616                            
    2      1.561267   1.556618                            
    3      1.538398   1.532233                            
    4      1.501769   1.500734                            
    5      1.463071   1.464634                            
    6      1.429594   1.449414                            
    7      1.422476   1.474208                            
    8      1.410068   1.45594                             
    9      1.388176   1.422122                            
    10     1.362936   1.409279                            
    11     1.337722   1.398316                            
    12     1.313663   1.379012                            
    13     1.291804   1.366855                            
    14     1.276164   1.363234                            



[1.3632339]

In [60]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm2_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**3-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      1.283274   1.359252  
    1      1.292899   1.390834                            
    2      1.272704   1.35908                             
    3      1.275871   1.36774                             
    4      1.265855   1.354711                            
    5      1.24568    1.339754                            
    6      1.223143   1.325305                            



[1.325305]

In [61]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm2_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**3-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      1.22147    1.334714  
    1      1.226097   1.33458                             
    2      1.205635   1.31099                             
    3      1.213911   1.34077                             
    4      1.208066   1.33293                             
    5      1.191204   1.313942                            
    6      1.168864   1.309636                            



[1.3096362]

## LSTM 3 Layer

In [16]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 3).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [17]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [18]:
fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      3.146723   3.110797  
    1      3.091744   3.102795                            



[3.1027951]

In [19]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm3_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      3.054288   3.095759  
    1      3.015687   2.953661                            
    2      3.002782   3.02681                             
    3      2.985458   2.882667                            
    4      2.885019   2.637842                            
    5      2.756208   2.525089                            
    6      2.657809   2.497811                            
    7      2.60508    2.359902                            
    8      2.474082   2.245984                            
    9      2.344696   2.113433                            
    10     2.235014   2.061319                            
    11     2.145487   2.002588                            
    12     2.076245   1.96289                             
    13     2.02372    1.947915                            
    14     1.988819   1.943067                            



[1.9430666]

In [20]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm3_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      2.098934   2.024783  
    1      2.018921   1.94091                             
    2      1.952348   1.895071                            
    3      1.915263   1.841914                            
    4      1.867101   1.793236                            
    5      1.818515   1.75873                             
    6      1.782191   1.753045                            
    7      1.763887   1.742456                            
    8      1.734667   1.704758                            
    9      1.700723   1.66803                             
    10     1.663924   1.627016                            
    11     1.628983   1.599626                            
    12     1.597032   1.585173                            
    13     1.572126   1.576489                            
    14     1.553207   1.56914                             
    15     1.567766   1.588259                            
    16     1.554906   1

[1.2933642]

In [28]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/lstm3_cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      1.195893   1.31311   
    1      1.183094   1.331738                            
    2      1.157545   1.288974                            
    3      1.166312   1.310036                            
    4      1.164878   1.322561                            
    5      1.146489   1.304674                            
    6      1.12477    1.284304                            
    7      1.137957   1.32084                             
    8      1.149119   1.329151                            
    9      1.14594    1.313118                            
    10     1.136527   1.291005                            
    11     1.12207    1.284786                            
    12     1.105203   1.284341                            
    13     1.089541   1.281934                            
    14     1.074024   1.285759                            



[1.2857587]

### Test

In [21]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [23]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

### LSTM 2 Layers Test

In [79]:
load_model(m,  f'{PATH}models/lstm2_cyc_0')

In [80]:
get_next('for thos')

'e'

In [85]:
print(get_next_n('t', 400))

turn thy rememper and rove the gods of the sking, to leave thanks, and therein ad thy wing shall lendeth: ditsmeng those tear me.22:10 the lord is name shall god? cassays shallbread guinem;38:22 therefore are with me: and strong is life out is mine eyely.72:16 let the lord will i heard thee beloved thy fattened him i am unto him, and all the waters of the declarts of glad for his congregation.46:9 


In [86]:
print(get_next_n('woe', 400))

woed, anus he spire outfeth oil fea; a preth wing hangs.27 thery, len heares with therver agan hisplan. sworve, o me terathey over: my throps agy.51 threis throws agan he to in me, as is mysture onf, the put trok thath stres, helved me?49:7 foar: lear thour god high: dign hear my our praing.84:18 throt; thour to my goous,4in upreword he poor; a nein thour from.3:21 than agano feeth higs tes it, times


In [83]:
print(get_next_n('praise the', 400))

praise the heavens.37:28 my send for me and thou hast sking?84:18 only soul, unaed him.12:1 purely in the pacpition of iniquity.31:7 thy lips of the sear of his holy bouth, and my housh.16:5 my god, and the voice of: let not tom the sight.67:12 thou spek not thee, o lord, and at about.96:5 they shall shall fish high.52:6 let them of righteousness shall death at heals past: the med shall be provoked them non


In [84]:
print(get_next_n('shadow', 400))

shadow.18:13 their right, withthere were fee, i call of that a tened.37:9 the love it: a favest ye his people anot shine abom.52:1 o lord: as forsable.18:5 his soul in thyselves; yetfrom the wrath.24:18 thou wilt before holiness; the heathe from him, givehh is god; for itaken the sanctuary, fear the refuge wroth from the lord. selah.56:7 day also art to burne that satis thee in his pringromintion.75:8 a


### LSTM 3 Layers Test

In [47]:
load_model(m,  f'{PATH}models/lstm3_cyc_5')

In [48]:
get_next('for thos')

'e'

In [51]:
print(get_next_n('t', 400))

the lord; he is the heapt; by menine iniquity be therein.59:11 for his heads ofthe lord cast them that fear the living, thou madest to forth.31:25 bless the never, over le with me, my clrangess unto the ungodly; his coveran,but there are to compassed:8:18 arise, o known that he shall thoubingeth, o lord that: he hath not been poor understeps of him: leldeth all not not are strength.33:5 give all th


In [50]:
print(get_next_n('woe', 400))

woed thour heat? throw moastear tabe;8 askingity a reingdens. end; trus outation: for thy upors, amony.32 with untrean outh: leth heave migy, the shat themble; amon brok; hear than.31 thembly cuto ened: i wassion thee:4 han in all:t good bless in themgest upon; with fultored a uning umoning. the in con: trour tuded: the repeth own thyholn.3 froaneth ters; i saintuster.31 won in hight theely, dwelt, a


In [52]:
print(get_next_n('praise the', 400))

praise them, rewill ip.41:6 let us to swallow.57: side to keep;79:10 he would a bowed himself in his name'fover thebeance of thehigh they culthepressicked unto the years withtheir destroyed.107:28 my heart, and together, and flourisy.90:4 but thou art mysuffering.112:9 trusteth for ever.107:29 at thee, o myhony blesses: i will preserved.89:9 because the people, judge me.20:3 for thou artlyfall his rugh of t


In [55]:
print(get_next_n('shadow', 400))

shadow by thyaffliction israel.72:4 then ye thy name? shall in thee.52:4 back to sake; all ye me: for heart.3 let all they hath magnify the with the horsepre.55:4 god the long: and shall before unto my distressor him.15:1 yea, they rejoice.109:4 but among into that sword:selah.79:49 king and tream:24:8 because off.73 makewituth: for with altogeth from lift.4 their equel:47 he blotter; the praise to all 
